# Simple EDA

In [1]:
import pandas as pd
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import count
from snowflake.snowpark.context import get_active_session

/home/roman/Dev/Python/snowflake_project/.venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)
/home/roman/Dev/Python/snowflake_project/.venv/lib/python3.12/site-packages/snowflake/snowpark/session.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
session = get_active_session()
df = session.table("RAW.CUSTOMER_CHURN_RAW")

# Basic info (logged, but not returned)
print("📊 Row count:", df.count())
df.print_schema()

print("\n🔍 Sample data:")
df.show(10)

# Categorical value counts
print("\n🧑‍🤝‍🧑 Gender distribution:")
df.group_by("GENDER").agg(count("*").alias("count")).show()

print("\n🌍 Geography distribution:")
df.group_by("GEOGRAPHY").agg(count("*").alias("count")).show()

print("\n💳 Card types:")
df.group_by("CARD_TYPE").agg(count("*").alias("count")).show()

# Binary flags
print("\n💳 Has credit card:")
df.group_by("HASCRCARD").agg(count("*").alias("count")).show()

print("\n🔥 Active members:")
df.group_by("ISACTIVEMEMBER").agg(count("*").alias("count")).show()

# Target variable
print("\n🚪 Exited distribution:")
df.group_by("EXITED").agg(count("*").alias("count")).show()

# Numerical summary
print("\n📈 Summary statistics (AGE, BALANCE, CREDITSCORE):")
df.select("AGE", "BALANCE", "CREDITSCORE").describe().show()

print("\n💰 Salary vs. Satisfaction:")
df.select("ESTIMATEDSALARY", "SATISFACTION_SCORE").describe().show()


SnowparkSessionException: (1403): No default Session is found. Please create a session before you call function 'udf' or use decorator '@udf'.